In [1]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import seaborn as sns
import random

def configure_model(model):
    model.Params.OutputFlag = 0
    model.Params.TimeLimit = 100
    model.Params.MIPGap = 0.05
    model.Params.Presolve = 2
    model.Params.Method = 1         # 让 Gurobi 自选（通常最平衡）
    model.Params.MIPFocus = 1        # 更关注可行解（非全局最优）
    model.Params.Heuristics = 0.8    # 启发式加速
    model.Params.Threads = 8


# 参数
m = 15
K = 5
L_levels = [20, 15, 10, 5, 1]  
E_levels = [13, 28, 38, 45, 120] 
E_PU_max = [8000] * m # limit 650
CPU_capacity = [9000] * m
f_v_levels = [15, 20, 30, 40, 60]
P_idle = [18] * m
P_max = [36] * m
T_total = 300
total_tasks = 100



chain_tasks = {
    0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    1: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
    2: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
    3: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
    4: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
    5: [50, 51, 52, 53, 54, 55, 56, 57],
    6: [58, 59, 60],
    7: [61, 62, 63],
    8: [64, 65, 66],
    9: [67, 68, 69],
    10: [70, 71, 72],
    11: [73, 74, 75],
    12: [76, 77, 78],
    13: [79, 80, 81],
    14: [82, 83, 84],
    15: [85, 86, 87],
    16: [88, 89, 90],
    17: [91, 92, 93],
    18: [94, 95, 96],
    19: [97, 98, 99]
}


num_chains = len(chain_tasks)
n = max(max(v) for v in chain_tasks.values()) + 1  # 总任务数
L_max = [len(chain_tasks[k]) * 6.5 for k in chain_tasks]  # 每条链的最大延迟




# L_max = [int(length * 6.5) for length in tasks_per_chain_list]


def configure_model(model):
    model.Params.OutputFlag = 0
    model.Params.TimeLimit = 300
    model.Params.MIPGap = 0.05
    model.Params.MIPFocus = 1
    model.Params.Presolve = 2
    model.Params.Method = 2
    model.Params.Threads = 8

def run_gurobi_model(alpha_1, alpha_2, alpha_3, baseline=False):
    model = gp.Model("MAF_Optimization")
    configure_model(model)
   

    delta = model.addVars(n, vtype=GRB.BINARY, name="delta")
    w = model.addVars(n, m, vtype=GRB.BINARY, name="w")
    y = model.addVars(n, K, vtype=GRB.BINARY, name="y")
    z = model.addVars(num_chains, vtype=GRB.BINARY, name="z")
    E_PU = model.addVars(m, name="E_PU")

    if baseline:
        for i in range(n):
            model.addConstr(y[i, K - 1] == delta[i])
            for k in range(K - 1):
                model.addConstr(y[i, k] == 0)

    for i in range(n):
        model.addConstr(gp.quicksum(w[i, j] for j in range(m)) == delta[i])
        model.addConstr(gp.quicksum(y[i, k] for k in range(K)) == delta[i])
        for k in range(K):
            model.addConstr(y[i, k] <= delta[i])  # ✅ 确保未激活任务 y[i,k] = 0

    for k in range(num_chains):
        model.addConstr(gp.quicksum(delta[i] for i in chain_tasks[k]) == z[k] * len(chain_tasks[k]))
        delay_chain = gp.quicksum(y[i, k_] * L_levels[k_] for i in chain_tasks[k] for k_ in range(K))
        model.addConstr(delay_chain <= L_max[k])


    for j in range(m):
        model.addConstr(
            gp.quicksum(w[i, j] * gp.quicksum(y[i, k] * f_v_levels[k] for k in range(K)) for i in range(n)) <= CPU_capacity[j]
        )
        active_time = gp.quicksum(w[i, j] * gp.quicksum(y[i, k] * L_levels[k] for k in range(K)) for i in range(n))
        U_j = active_time / T_total
        E_PU_expr = P_idle[j] * (T_total - active_time) + (P_max[j] - P_idle[j]) * U_j * T_total
        model.addConstr(E_PU[j] == E_PU_expr)
        model.addConstr(E_PU[j] <= E_PU_max[j])

    total_task_energy = gp.quicksum(y[i, k] * E_levels[k] for i in range(n) for k in range(K))
    total_chain_accept = gp.quicksum(z[k] for k in range(num_chains))
    total_pu_energy = gp.quicksum(E_PU[j] for j in range(m))

    model.setObjective(
        alpha_1 * total_chain_accept - alpha_2 * total_task_energy - alpha_3 * total_pu_energy,
        GRB.MAXIMIZE
    )


    start_time = time.time()
    model.optimize()
    end_time = time.time()
    if model.status != GRB.OPTIMAL:
        print("❌ 模型未成功优化，跳过变量访问")
        return None
    print(f"⏱️ Gurobi 运行时间: {end_time - start_time:.2f} 秒")
    print("\n📋 链激活状态总览：")
    for k in range(num_chains):
        status = "✅ 激活" if z[k].X > 0.5 else "❌ 未激活"
        task_list = chain_tasks[k]
        print(f"链 {k}: {status}, 包含任务: {task_list}")
    
    accepted_chains = [k for k in range(num_chains) if z[k].X > 0.5]
    pu_assignment = [j for i in range(n) for j in range(m) if w[i, j].X > 0.5]
    pu_task_map = {}
    for task_id, pu_id in enumerate(pu_assignment):
         pu_task_map.setdefault(pu_id, []).append(task_id)
    
    task_energy_levels = [max((y[i, k].X, k) for k in range(K))[1] if delta[i].X > 0.5 else -1 for i in range(n)]



    return {
        "模式": "Baseline" if baseline else "Non-Baseline",
        "目标函数值": model.ObjVal,
        "被接受的任务链数": len(accepted_chains),
        "PU 总能耗": sum(E_PU[j].X for j in range(m)),
        "任务总能耗": sum(y[i, k].X * E_levels[k] for i in range(n) for k in range(K)),
        "运行时间": model.Runtime,
        "PU 任务分配": pu_task_map,
        "任务能源等级": task_energy_levels,
        "激活链列表": [1 if z[k].X > 0.5 else 0 for k in range(num_chains)]  # ✅ 新增
    }

In [2]:
chain_id = 0
num_tasks = len(chain_tasks[chain_id])
cpu_per_task = f_v_levels[-1]  # 等级4 = 60
energy_per_task = E_levels[-1]  # 等级4 = 120

print(f"链 {chain_id} 任务数 = {num_tasks}")
print(f"总 CPU 需求 = {num_tasks * cpu_per_task}")
print(f"总任务能耗 = {num_tasks * energy_per_task}")

链 0 任务数 = 10
总 CPU 需求 = 600
总任务能耗 = 1200


In [4]:
baseline_result = run_gurobi_model(alpha_1=3500, alpha_2=5, alpha_3=1, baseline=True)

⏱️ Gurobi 运行时间: 0.00 秒

📋 链激活状态总览：
链 0: ❌ 未激活, 包含任务: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
链 1: ❌ 未激活, 包含任务: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
链 2: ❌ 未激活, 包含任务: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
链 3: ❌ 未激活, 包含任务: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
链 4: ❌ 未激活, 包含任务: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
链 5: ❌ 未激活, 包含任务: [50, 51, 52, 53, 54, 55, 56, 57]
链 6: ✅ 激活, 包含任务: [58, 59, 60]
链 7: ✅ 激活, 包含任务: [61, 62, 63]
链 8: ✅ 激活, 包含任务: [64, 65, 66]
链 9: ✅ 激活, 包含任务: [67, 68, 69]
链 10: ✅ 激活, 包含任务: [70, 71, 72]
链 11: ✅ 激活, 包含任务: [73, 74, 75]
链 12: ✅ 激活, 包含任务: [76, 77, 78]
链 13: ✅ 激活, 包含任务: [79, 80, 81]
链 14: ✅ 激活, 包含任务: [82, 83, 84]
链 15: ✅ 激活, 包含任务: [85, 86, 87]
链 16: ✅ 激活, 包含任务: [88, 89, 90]
链 17: ✅ 激活, 包含任务: [91, 92, 93]
链 18: ✅ 激活, 包含任务: [94, 95, 96]
链 19: ✅ 激活, 包含任务: [97, 98, 99]


In [6]:
import pandas as pd

# 使用用户提供的真实 run_gurobi_model 测试 alpha_2 扫描
alpha_1 = 14000
alpha_3 = 1.5
alpha_2_list = [25, 29, 35, 40, 50]

# 存储结果
results = []

for alpha_2 in alpha_2_list:
    print(f"\n🚀 alpha_2 = {alpha_2} | Baseline")
    baseline_result = run_gurobi_model(alpha_1, alpha_2, alpha_3, baseline=True)
    if baseline_result:
        baseline_result["alpha_2"] = alpha_2
        results.append(baseline_result)

    print(f"\n🚀 alpha_2 = {alpha_2} | Optimal")
    optimal_result = run_gurobi_model(alpha_1, alpha_2, alpha_3, baseline=False)
    if optimal_result:
        optimal_result["alpha_2"] = alpha_2
        results.append(optimal_result)

# 整理成 DataFrame 以便展示和分析
df = pd.DataFrame(results)




🚀 alpha_2 = 25 | Baseline
⏱️ Gurobi 运行时间: 0.01 秒

📋 链激活状态总览：
链 0: ❌ 未激活, 包含任务: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
链 1: ❌ 未激活, 包含任务: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
链 2: ❌ 未激活, 包含任务: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
链 3: ❌ 未激活, 包含任务: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
链 4: ❌ 未激活, 包含任务: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
链 5: ❌ 未激活, 包含任务: [50, 51, 52, 53, 54, 55, 56, 57]
链 6: ✅ 激活, 包含任务: [58, 59, 60]
链 7: ✅ 激活, 包含任务: [61, 62, 63]
链 8: ✅ 激活, 包含任务: [64, 65, 66]
链 9: ✅ 激活, 包含任务: [67, 68, 69]
链 10: ✅ 激活, 包含任务: [70, 71, 72]
链 11: ✅ 激活, 包含任务: [73, 74, 75]
链 12: ✅ 激活, 包含任务: [76, 77, 78]
链 13: ✅ 激活, 包含任务: [79, 80, 81]
链 14: ✅ 激活, 包含任务: [82, 83, 84]
链 15: ✅ 激活, 包含任务: [85, 86, 87]
链 16: ✅ 激活, 包含任务: [88, 89, 90]
链 17: ✅ 激活, 包含任务: [91, 92, 93]
链 18: ✅ 激活, 包含任务: [94, 95, 96]
链 19: ✅ 激活, 包含任务: [97, 98, 99]

🚀 alpha_2 = 25 | Optimal
⏱️ Gurobi 运行时间: 0.36 秒

📋 链激活状态总览：
链 0: ✅ 激活, 包含任务: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
链 1: ✅ 激活, 包含任务: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
链 2: 

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

# 假设之前已经成功运行模型，下面是模拟的数据用于绘图展示
# 实际中应替换为 run_gurobi_model 的真实输出

# 模拟数据结构（每项为一个模型运行结果）
alpha_1_list = [12000, 14000, 16000, 18000]
alpha_2 = 30
alpha_3 = 1.5

# 存储结果
results = []

for alpha_1 in alpha_1_list:
    print(f"\n🚀 alpha_1 = {alpha_1} | Baseline")
    baseline_result = run_gurobi_model(alpha_1, alpha_2, alpha_3, baseline=True)
    if baseline_result:
        baseline_result["alpha_1"] = alpha_1
        results.append(baseline_result)

    print(f"\n🚀 alpha_1 = {alpha_1} | Optimal")
    optimal_result = run_gurobi_model(alpha_1, alpha_2, alpha_3, baseline=False)
    if optimal_result:
        optimal_result["alpha_1"] = alpha_1
        results.append(optimal_result)

# 整理成 DataFrame 以便展示和分析
df = pd.DataFrame(results)


🚀 alpha_1 = 12000 | Baseline
⏱️ Gurobi 运行时间: 0.00 秒

📋 链激活状态总览：
链 0: ❌ 未激活, 包含任务: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
链 1: ❌ 未激活, 包含任务: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
链 2: ❌ 未激活, 包含任务: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
链 3: ❌ 未激活, 包含任务: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
链 4: ❌ 未激活, 包含任务: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
链 5: ❌ 未激活, 包含任务: [50, 51, 52, 53, 54, 55, 56, 57]
链 6: ✅ 激活, 包含任务: [58, 59, 60]
链 7: ✅ 激活, 包含任务: [61, 62, 63]
链 8: ✅ 激活, 包含任务: [64, 65, 66]
链 9: ✅ 激活, 包含任务: [67, 68, 69]
链 10: ✅ 激活, 包含任务: [70, 71, 72]
链 11: ✅ 激活, 包含任务: [73, 74, 75]
链 12: ✅ 激活, 包含任务: [76, 77, 78]
链 13: ✅ 激活, 包含任务: [79, 80, 81]
链 14: ✅ 激活, 包含任务: [82, 83, 84]
链 15: ✅ 激活, 包含任务: [85, 86, 87]
链 16: ✅ 激活, 包含任务: [88, 89, 90]
链 17: ✅ 激活, 包含任务: [91, 92, 93]
链 18: ✅ 激活, 包含任务: [94, 95, 96]
链 19: ✅ 激活, 包含任务: [97, 98, 99]

🚀 alpha_1 = 12000 | Optimal
⏱️ Gurobi 运行时间: 0.62 秒

📋 链激活状态总览：
链 0: ❌ 未激活, 包含任务: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
链 1: ❌ 未激活, 包含任务: [10, 11, 12, 13, 14, 15, 16, 17, 18, 1